In [1]:
import h5py
import numpy as np
import glob

In [2]:
f = h5py.File('/global/cfs/projectdirs/desi/users/akim/Projects/QSO_Distance/data/dates.hdf5')

In [3]:
for name in f['df']['block0_items']:
    print(name)
print('//////////')
for name in f['df']['block1_items']:
    print(name)
print('//////////')

b'mjd'
b'z'
b'zerr'
b'zwarn'
//////////
b'targetid'
b'night'
//////////


In [4]:
f1 = h5py.File('/global/cfs/projectdirs/desi/users/akim/Projects/QSO_Distance/data/lc/001786.hdf5')

In [5]:
#for name in f1:
    #print(name)
for name in f1['39633207531932739']:
    print(name)

g
i
r


In [6]:
file_pattern = '/global/cfs/projectdirs/desi/users/akim/Projects/QSO_Distance/data/lc/*.hdf5'
files = glob.glob(file_pattern)

In [7]:
target_id_g = []
mag_g = []
magerr_g = []
mjd_g = []

target_id_r = []
mag_r = []
magerr_r = []
mjd_r = []

target_id_i = []
mag_i = []
magerr_i = []
mjd_i = []

for file_path in files:
    with h5py.File(file_path, 'r') as hdf:
        keys = list(hdf.keys())
        for key in keys:
            if 'g' in hdf[key] and 'mag' in hdf[key]['g'] and 'magerr' in hdf[key]['g']:
                indices = (hdf[key]['g']['catflags'][:] & 32768) == 0
                target_id_g.append(key)
                mag_g.append(hdf[key]['g']['mag'][indices])
                magerr_g.append(hdf[key]['g']['magerr'][indices])
                mjd_g.append(hdf[key]['g']['hmjd'][indices])
            if 'r' in hdf[key] and 'mag' in hdf[key]['r'] and 'magerr' in hdf[key]['r']:
                indices = (hdf[key]['r']['catflags'][:] & 32768) == 0
                target_id_r.append(key)  # Correctly append to 'r' lists
                mag_r.append(hdf[key]['r']['mag'][indices])
                magerr_r.append(hdf[key]['r']['magerr'][indices])
                mjd_r.append(hdf[key]['r']['hmjd'][indices])
            if 'i' in hdf[key] and 'mag' in hdf[key]['i'] and 'magerr' in hdf[key]['i']:
                indices = (hdf[key]['i']['catflags'][:] & 32768) == 0
                target_id_i.append(key)  # Correctly append to 'i' lists
                mag_i.append(hdf[key]['i']['mag'][indices])
                magerr_i.append(hdf[key]['i']['magerr'][indices])
                mjd_i.append(hdf[key]['i']['hmjd'][indices])

In [8]:
print(len(target_id_g))
print(len(mag_g))

1033156
1033156


In [9]:
# m is the magnitude, dm is the uncertainty of magnitude,f is flux, df is the uncertainty of flux
def mag_flux(m,dm):
    f = 10 ** (-m / 2.5)
    df = np.abs(f * dm * np.log(10) / (-2.5))
    return f,df
def weighted_average(f,df):
    weights = 1.0 / (df ** 2)
    weighted_f = np.sum(f * weights) / np.sum(weights)
    weighted_df = np.sqrt(1.0 / np.sum(weights))
    return weighted_f,weighted_df

In [10]:
result1 = [mag_flux(mag_g[x],magerr_g[x]) for x in range(len(mag_g))]
result2 = [mag_flux(mag_r[x],magerr_r[x]) for x in range(len(mag_r))]
result3 = [mag_flux(mag_i[x],magerr_i[x]) for x in range(len(mag_i))]

f_g = [x[0] for x in result1]
df_g = [x[1] for x in result1]

f_r = [x[0] for x in result2]
df_r = [x[1] for x in result2]

f_i = [x[0] for x in result3]
df_i = [x[1] for x in result3]

In [11]:
new_result1 = [weighted_average(f_g[x],df_g[x]) for x in range(len(f_g)) ]
new_result2 = [weighted_average(f_r[x],df_r[x]) for x in range(len(f_r)) ]
new_result3 = [weighted_average(f_i[x],df_i[x]) for x in range(len(f_i)) ]



#print(len(result))
avg_f_g = [x[0] for x in new_result1]
avg_df_g = [x[1] for x in new_result1]

avg_f_r = [x[0] for x in new_result2]
avg_df_r = [x[1] for x in new_result2]

avg_f_i = [x[0] for x in new_result3]
avg_df_i = [x[1] for x in new_result3]
#print(weighted_ave_mag_g)
f_g = np.array([avg_f_g])
df_g = np.array([avg_df_g])

f_r = np.array([avg_f_r])
df_r = np.array([avg_df_r])

f_i = np.array([avg_f_i])
df_i = np.array([avg_df_i])

mag_g = -2.5 * np.log10(f_g)
mag_r = -2.5 * np.log10(f_r)
mag_i = -2.5 * np.log10(f_i)
magerr_g = np.abs(2.5 / np.log(10) * df_g / f_g)
magerr_r = np.abs(2.5 / np.log(10) * df_r / f_r)
magerr_i = np.abs(2.5 / np.log(10) * df_i / f_i)

In [12]:
new_mjd_g = [np.mean(mjd_g[x]) for x in range(len(mjd_g))]
new_mjd_r = [np.mean(mjd_r[x]) for x in range(len(mjd_r))]
new_mjd_i = [np.mean(mjd_i[x]) for x in range(len(mjd_i))]

In [13]:
print(mag_g)

[[21.559174 21.5149   21.663918 ... 20.473686 20.330414 21.48779 ]]


In [14]:
from astropy.table import Table

In [15]:
print(type(target_id_g[0]))
print(mag_g[0].dtype)

<class 'str'>
float32


In [16]:
new_target_id_g = np.array(target_id_g)

In [17]:
print(new_target_id_g.shape)

(1033156,)


In [18]:
t = Table([target_id_g, mag_g[0], magerr_g[0],new_mjd_g], names=('target_id_g', 'mag_g', 'magerr_g','mjd_g'))

t.write('/pscratch/sd/z/zhaozhon/DATASET/ZTF-data/g.csv', format='csv', overwrite=True)

In [19]:
t = Table([target_id_r, mag_r[0], magerr_r[0],new_mjd_r], names=('target_id_r', 'mag_r', 'magerr_r','mjd_r'))

t.write('/pscratch/sd/z/zhaozhon/DATASET/ZTF-data/r.csv', format='csv', overwrite=True)

In [20]:
t = Table([target_id_i, mag_i[0], magerr_i[0],new_mjd_i], names=('target_id_i', 'mag_i', 'magerr_i','mjd_i'))

t.write('/pscratch/sd/z/zhaozhon/DATASET/ZTF-data/i.csv', format='csv', overwrite=True)